In [31]:
import numpy as np
# importing decimal module to control significant figures
import decimal

In [32]:
def toDecimal(array) :
    # flatten the array convert floats to decimal and reshape to original shape
    originalShape = array.shape
    array = array.flatten().astype(object) 
    for i in range(array.size) :
        array[i] = decimal.Decimal(str(array[i]))
    array = array.reshape(originalShape)   
    return array


def intializeContext(significantFigs,rounding) :
    #Intialize the context with specified signFigs and rounding/chopping
    decimal.getcontext().prec = significantFigs 
    if( rounding ) :
        decimal.getcontext().rounding = decimal.ROUND_HALF_UP
    else :
        decimal.getcontext().rounding = decimal.ROUND_DOWN 
        
def toFloats (array) :
    #Convert the decimals back to float
    originalShape = array.shape
    array = array.flatten().astype(object)
    for i in range(array.size) :
        array[i] = float(array[i]) 
    array = array.reshape(originalShape)    
    return array            
    

In [33]:
def isDiagonalyDominant(A) :
    #Check if Matrix A is diagonaly dominant 
    dim = A.shape
    # Flags to check system
    at_least_one_strictly_greater = None 
    if(dim[0] != dim[1]) : raise Exception("Matrix must be square")
    for i in range(dim[0]) :
        nonDiagonalSum = 0 
        for j in range(dim[0]) :
            if(j==i) : continue 
            nonDiagonalSum += abs(A[i][j]) 
        if(A[i][i] > nonDiagonalSum) :
            at_least_one_strictly_greater = True
        elif(A[i][i] == nonDiagonalSum) :
            if(at_least_one_strictly_greater == None) : at_least_one_strictly_greater = False
        else :
            return False    
    return at_least_one_strictly_greater            
            

In [ ]:
# Jacobi with relaxation for improved convergence (called weighted Jacobi)
# the parameter n can be removed as we can get the no of vars from the npArray
def Jacobi_noNorm(A,b,n,x,maxIterations,ErrorTolerance,relax , significantFigs = 7 , rounding = True) :
    # Setting up signifcant figs and rounding/chopping
    intializeContext(significantFigs,rounding)
    #Copying the arrays to avoid modifying original arrays
    A = A.copy()
    b = b.copy()
    x = x.copy()
    #converting floats to decimals 
    A = toDecimal(A)
    b = toDecimal(b)
    x = toDecimal(x)
    relax = decimal.Decimal(str(relax))
    #Array to hold new values
    xNew = np.zeros(n,dtype=object)
    #List to hold iteration details
    iteration_details = []
    # Calculating first iteration before applying relaxation    
    for i in range(n) :
        sum = b[i]
        for j in range(n) :
            if(i==j) :
                continue
            sum -= A[i][j] * x[j]
        xNew[i] = sum/A[i][i]
    x = xNew.copy()
     # Storing details of first iteration    
    details = {
            'iteration' : 1,
            'xNew' : toFloats(xNew),
            'maxError' : '_'
        }     
    iteration_details.append(details)
    iteration = 2
    # Loop until convergence or max iterations reached 
    while (True) :
        belowTolerance = True
        maxError = decimal.Decimal('0')
        for i in range(n) :
            oldX = x[i]
            sum = b[i]
            for j in range(n) :
                if(i==j) :
                    continue
                sum -= A[i][j] * x[j]
            xNew[i] = relax*sum/A[i][i] + (1-relax)*oldX
            if (belowTolerance and xNew[i] != 0) :
                estimatedError = abs((xNew[i]-oldX)/xNew[i]) * 100
                maxError = max(maxError, estimatedError)
                if(estimatedError > ErrorTolerance):
                    belowTolerance = False
        details = {
            'iteration' : iteration,
            'xNew' : toFloats(xNew),
            'maxError' : float(maxError)
        }         
        iteration_details.append(details)
        iteration+=1
        x = xNew.copy()
        if(belowTolerance or iteration >= maxIterations) :
            break
    return toFloats(xNew) , iteration_details

In [35]:
# =========================
# Test the Jacobi_noNorm function
# =========================

# Example linear system
A = np.array([[10, -1, 2, 0],
              [-1, 11, -1, 3],
              [2, -1, 10, -1],
              [0, 3, -1, 8]], dtype=float)

b = np.array([6, 25, -11, 15], dtype=float)

# Initial guess
x0 = np.zeros(len(b))

# Run weighted Jacobi
solution, details = Jacobi_noNorm(A, b, 4, x0, maxIterations=50, ErrorTolerance=1e-6, relax=0.8)

print("Solution:", solution)
print("\nIteration Details:")
for d in details:
    max_error = d['maxError']
    # Ensure maxError is a float, else print as-is
    if isinstance(max_error, (float, int)):
        max_error_str = f"{max_error:.6f}"
    else:
        max_error_str = str(max_error)
    print(f"Iteration {d['iteration']}: x = {d['xNew']}, maxError = {max_error_str}")


Solution: [0.9999996 1.999999 -0.9999997 1.000001]

Iteration Details:
Iteration 1: x = [0.6 2.272727 -1.1 1.875], maxError = _
Iteration 2: x = [0.9578184 1.827272 -0.8641818 1.083182], maxError = 0.373576
Iteration 3: x = [0.9560146 1.954115 -0.9732516 1.082037], maxError = 0.001887
Iteration 4: x = [0.9832523 1.97167 -0.9847209 1.032848], maxError = 0.027702
Iteration 5: x = [0.9919395 1.98706 -0.9939036 1.016597], maxError = 0.008758
Iteration 6: x = [0.9963773 1.993648 -0.9971985 1.007811], maxError = 0.004454
Iteration 7: x = [0.9983191 1.996966 -0.9987436 1.003748], maxError = 0.001945
Iteration 8: x = [0.99922 1.998545 -0.9994225 1.001785], maxError = 0.000902
Iteration 9: x = [0.9996352 1.999305 -0.9997337 1.000851], maxError = 0.000415
Iteration 10: x = [0.9998288 1.999669 -0.9998762 1.000405], maxError = 0.000194
Iteration 11: x = [0.9999195 1.999842 -0.999942 1.000193], maxError = 0.000091
Iteration 12: x = [0.999962 1.999924 -0.999973 1.000092], maxError = 0.000043
Iterati